In [ ]:
import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import math
from typing import List
from types import SimpleNamespace
from tqdm import tqdm

**Fill in your name and your partner's name below** (and name the `.ipynb` file correctly):

---

### JUELS Titan Harold

### YOUR PARTNER'S NAME

---

# Exercise 4 - ML & Neural Networks  (DD1420)

## Instructions

This Jupyter Notebook contains exercises for DD1420 **Module 2: Machine Learning & Optimization**. To receive credit, all cells must be completed and must function correctly.

### Collaboration

- **Collaboration**: You are allowed to work on this exercise in pairs. However, each student must submit their own completed notebook. If you worked with a partner, please indicate your partner in the filename as described below.
- **Code of Conduct**: Adhere to the following:
  1. All group members are responsible for the work submitted.
  2. Each student must honestly disclose any assistance received or external sources used.
  3. Do not copy from other students' solutions. If you need help, use the appropriate Discussion Topic or sign up for a help session.

### Completing the Exercise

- **Code Cells**: Replace any placeholder comments like `YOUR CODE HERE` or "YOUR ANSWER HERE" with your own code or response. Once implemented, delete any instances of `raise NotImplementedError()`.
- **Library Imports**: Do not import any additional libraries beyond those already included in the assignment.
- **Derivations**: For derivation questions, you may use $\LaTeX$ in markdown cells or upload an image of your handwritten derivation. In *Google Colab*, to upload an image, create a text cell, click the `Insert Image` icon, and upload your file.

### Submission Instructions

1. **Final Check**: Before submitting, ensure everything runs correctly by selecting `Runtime` -> `Restart and Run All`.
2. **File Naming**: Download the notebook by selecting `File` -> `Download` -> `Download .ipynb`. Rename the file as follows:
  - `Ex??_YOURLASTNAME_YOURFIRSTNAME_and_PARTNERLASTNAME_PARTNERFIRSTNAME.ipynb`
  - Replace `??` with the correct exercise number. If you worked alone, omit the partner's name from the filename.
3. **Submission**: Submit the `.ipynb` file to Canvas.

### Oral Examination

- **Preparation**: During the oral examination, you will be asked to demonstrate and explain your work. Have the link to your `.ipynb` file ready, with all cells executed.



# 4.1 Training a neural network with backpropagation

In this exercise we will once again train a machine learning algorithm to classify handwritten digits from MNISt, although this time we will use a bigger version with $64 \times 64$ images and 60,000 examples.

<br>

We will make use of [PyTorch](https://pytorch.org/) which is a popular machine learning framework for developing neural networks. You will not be expected to write PyTorch-specific code in this exercise, just standard numpy and Python. You won't need to follow along with what PyTorch is doing - we try to explain the important bits.

<br>

The first thing we will do is load the MNIST handwritten digits dataset,

In [ ]:
# these are pytorch's dataset class. it downloads and sets up the dataset
mnist_train = torchvision.datasets.MNIST(root="data",
                                         train=True,
                                         download=True,
                                         transform=T.Compose([T.ToTensor(),
                                                              T.Lambda(lambda x: torch.flatten(x)),
                                                              ]),
                                         )
mnist_test = torchvision.datasets.MNIST(root="data",
                                        train=False,
                                        download=True,
                                        transform=T.Compose([T.ToTensor(),
                                                             T.Lambda(lambda x: torch.flatten(x)),
                                                             ]),
                                        )

We use something called a `dataloader` class from PyTorch, which handles feeding data from the dataset to the network. It does things like shuffle the data and yield samples.



In [ ]:
# these are pytorch's dataloader class. they shuffle and yield sample points
# for us. you don't have to code data manipulations yourself in this exercise
train_loader = DataLoader(mnist_train, batch_size=1, shuffle=True)
val_loader = DataLoader(mnist_test, batch_size=1, shuffle=True)

Next we will define the size of our neural network, which will have $L = 3$ layers of width $m_{\ell=\{1,\dots,L\}} = \{784, 30, 10\}$

In [ ]:
# hyperparameters. sets up the network dimensions
# how many nodes in each
layers_list = [784, 30, 10]
L = len(layers_list)

We will define a function that will rescale our random weights to a distribution that has been proven to be a better initialization than just a standard `randn` distribution. The Xavier initialization simply divides the randomly initialized weights of each layer by the square root of the input size $\sqrt{k}$.

In [ ]:
def initialization_scaling(weight_matrix):
  """
  This implements Xavier initialization. It helps training of deep networks.
  """
  input_size = weight_matrix.shape[0]
  return weight_matrix / math.sqrt(input_size)

**4.1.1** *Complete the function provided below that creates a single layer of a neural network. Given the input size and the ouptut size this function should create the weight matrix and bias vector with random initialization. If the `scaled_init` flag is `True` it should apply the Xavier scaled defined above. The bias vector should be initialized to zero.*

In [ ]:
def create_single_layer(input_size: int, output_size: int, scaled_init=False):
  """
  Create a single layer
  output: a layer object. weights and biases are in layer.weight and layer.bias

  Usage:
    layer = create_single_layer(784, 10)
    layer_weight_matrix = layer.weight
    layer_bias_vector = layer.bias
  """
  ### YOUR CODE HERE ###
  weight_matrix = np.random.rand(output_size, input_size)
  bias_vector = np.zeros(output_size).reshape(-1, 1)


  if scaled_init:
    weight_matrix = initialization_scaling(weight_matrix)

  # The output being a SimpleNamespace is just a coding convenience. It
  # emulates how we access the weights and biases in pytorch.
  # without this we would have to access weights using layers[0][0],
  # with this we can do layers[0].weight so that it is a bit more readable
  return SimpleNamespace(weight=weight_matrix, bias=bias_vector)

**4.1.2** *Complete the function provided below that creates all the layers of the neural network using the function you created in 4.1.1. The `layers_list` contains a list of layer widths for each layer. Loop through that list, create each layer, and append it to a `layers` list. The function should output `layers`, a list of initialized layers.*

In [ ]:
def create_layers(layers_list: List):
  """
  Creates the weights and biases if layers according to the given list
  output: a list of layers

  Usage:
    layers = create_layers(layers_list)
    some_layer = layers[np.random.randint(L)]
  """

  #To match the Lecture notation we start with idx=1, i.e. there is no W^(0) or b^(0)
  layers = [None]
  for idx in range(L-1):
      ### YOUR CODE HERE###
      curr_layer = create_single_layer(layers_list[idx], layers_list[idx + 1], True)
      layers.append(curr_layer)
  return layers

Next, we provide you with code for the logistic sigmoid activation function `sigmoid` and its derivative `sigmoid_prime` which should be applied to pre-activations.

In [ ]:
def sigmoid(z):
  """
  The sigmoid function
  Usage:
    activation_output = sigmoid(some_input)
  """
  return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
  """
  $\sigma\prime$ in the lecture notes
  """
  return sigmoid(z)*(1-sigmoid(z))

**4.1.3** *Complete the following functions that the define the MSE loss (the loss for a single sample $\ell_{MSE}$) and its derivative.*

Note that this is not a very good choice of a loss for classification, but the derivative is easier to calculate and code, and it will give reasonable results on MNIST.

In [ ]:
def loss_fn(y_hat, y):
  """
  This is the MSE loss for a single sample.
  """
  ### YOUR CODE HERE ###
  return (y - y_hat)**2


def dloss_dyhat(y_hat, y):
  """
  Gradient of the MSE loss for a single sample.
  """
  ### YOUR CODE HERE ###
  return 2*(y-y_hat)


Next, we will start to define some of the essential functions for training the neural network. We will start with a function for the pre-activations $z^{(\ell)}$ and the activations $a^{(\ell)}$.

**4.1.4**.  *Complete the function below that returns the preactivation given the layers weight matrix $W^{(\ell)}$, bias vector $b^{(\ell)}$ and the previous layers activation $a^{(\ell-1)}$.*

hint: [The @ operator can be used as a shorthand for np.matmul on ndarrays.](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html)

In [ ]:
def calculate_preactivation(layer_weight, layer_bias, previous_activation):
  ### YOUR CODE HERE ###
  z = layer_weight @ previous_activation + layer_bias
  return z

A quick sanity check.

In [ ]:
W = np.array([[2,-2], [-3,-1], [1, -3]])
b = np.array([[-1], [0], [0]])
a = np.array([[2],[-2]])
calculate_preactivation(W,b,a)

array([[ 7],
       [-4],
       [ 8]])

**4.1.5**.  *Complete the function below that returns the activation of a layer given the preactivation $z^{(\ell)}$.*

In [ ]:
def calculate_activation(z):
  ### YOUR CODE HERE ###
  return sigmoid(z)

A quick sanity check.

In [ ]:
z = np.array([[-1], [0], [1]])
calculate_activation(z)

array([[0.26894142],
       [0.5       ],
       [0.73105858]])

**4.1.6.** *Write a function that computes and returns the gradient of the last layer $g^{(L)}$. You should make use of some previous functions you have written. You may need to use `.squeeze()` to ensure dimensions match.*

In [ ]:
def calculate_g_L(y, a, z):
  # YOUR CODE HERE
  dy_hat = dloss_dyhat(a, y).squeeze()

  return dy_hat @ np.diag(sigmoid_prime(z).squeeze())

A quick sanity check.


In [ ]:
y = np.ones((5,1))
a = np.zeros((5,1))
z = np.ones((5,1))
calculate_g_L(y,a,z)


array([0.39322387, 0.39322387, 0.39322387, 0.39322387, 0.39322387])

**4.1.7.** *Write a function that computes and returns the gradient of an aribtrary layer $g^{(\ell)}$ (except $L$). You may need to use `.squeeze()` to ensure dimensions match.*

In [ ]:
def calculate_g_l(g_l_previous, layer_weight_previous, z):
  ### YOUR CODE HERE ###
  g_l_previous = np.array(g_l_previous).reshape(-1, 1)
  z = np.array(z).reshape(-1, 1)
  g_l = (layer_weight_previous.T @ g_l_previous) * sigmoid_prime(z)
  # return g_l_previous @ layer_weight_previous @ np.diag(sigmoid_prime(z).squeeze())
  return g_l

A quick sanity check.


In [ ]:
y = np.ones((5,1))
a = np.zeros((5,1))
z = np.ones((5,1))
g_L = calculate_g_L(y,a,z)
W = np.ones((5,5))
z = np.ones((5,1))
g_l = calculate_g_l(g_L,W,z)
g_l

array([[0.38656252],
       [0.38656252],
       [0.38656252],
       [0.38656252],
       [0.38656252]])

**4.1.8.** *Write a function that computes and returns the gradient of the loss w.r.t. the weights $\nabla_{W^{(\ell)}} J$ given the gradient $g^{(\ell)}$ and the activations from the next layer $a^{(\ell-1)}$.*

In [ ]:
def calculate_nabla_weight(g_l, a_next):
  ### YOUR CODE HERE ###
  g_l = np.array(g_l).reshape(-1, 1)
  a_next = np.array(a_next).reshape(-1, 1)
  return g_l @ a_next.T

A quick sanity check.

In [ ]:
g_l = np.ones((1,5))
a_next = np.ones((3,1))
nabW = calculate_nabla_weight(g_l, a_next)
nabW

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

**4.1.9.** *Write a function that computes and returns the gradient of the loss w.r.t. the bias $\nabla_{b^{(\ell)}} J$ given the gradient $g^{(\ell)}$ and the activations from the next layer $a^{(\ell-1)}$.*

In [ ]:
def calculate_nabla_bias(g_l):
  ### YOUR CODE HERE
  g_l = np.array(g_l).reshape(-1, 1)
  return g_l

A quick sanity check

In [ ]:
g_l = np.ones((1,5))
nabb = calculate_nabla_bias(g_l)
nabb

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

Great! We are finally ready to put everything together and train the neural network. We have provided most of the code for you.

<br>

**4.1.10.** *Insert the functions you have defined in **4.1.2** and **4.1.5** through **4.1.9** in the correct places in the following code. Then run the code to train the neural network to classify handwritten digits.*

In [ ]:
# initialization = create the layers
layers = create_layers(layers_list)
lr = 3.0
# this is one epoch
# ====== loop through 1 epoch using the data loader =====
for (x, y) in tqdm(train_loader, desc=f'Epoch progress'):
  # we used pytorch loaders, convert back to numpy and proper shape
  x = x.numpy().T
  # our network uses one_hot encoding to classify the digit so
  # we convert the target to a one_hot encoding
  y = torch.nn.functional.one_hot(y,num_classes=10).numpy().T
  # actual updates occur here
  activation = x
  # set up the variables for saving the a & z values
  # a^(0) is set to x
  activations = [activation]
  # z^(0) does not exist so we store a None for it
  pre_activations = [None]
  # ====== do forward propagation =====
  # looping through the layers, computing and storing activations & pre-activations
  for layer in layers[1:]: #since layers[0] does not exist/is None
    pre_activation = calculate_preactivation(layer.weight, layer.bias, activations[-1])
    activation = calculate_activation(pre_activation)
    # store these for the backprop
    pre_activations.append(pre_activation)
    activations.append(activation)

  # ===== do backpropagation =====
  # setting up record keeping of the gradients, nabla_weights and nabla_biases
  # note that we have no gradients for W^(0) since we index starting at 1
  g_ls = [None]
  nabla_weights = [None]
  nabla_biases = [None]
  g_ls.extend([np.zeros((1, m)) for m in layers_list] )
  nabla_weights.extend([np.zeros_like(layer.weight) for layer in layers[1:]])
  nabla_biases.extend([np.zeros_like(layer.bias) for layer in layers[1:]])

  # looping through the layers, compute gradients and nabla's
  for l in range(L-1, 0, -1):
    # g^L is calculated differently, so we check and handle that first
    if l == L-1:
      g_ls[l] = calculate_g_L(y, activation[l], pre_activations[l])
    else:
      g_ls[l] = calculate_g_l(g_ls[l + 1], layers[l + 1].weight, pre_activation[l])
    nabla_weights[l] = calculate_nabla_weight(g_ls[l], activations[l - 1])
    nabla_biases[l] = calculate_nabla_bias(g_ls[l])

  # update the weights with the lr according to SGD algorithm
  for l in range(L-1, 0, -1):
    layers[l].weight = layers[l].weight - lr * nabla_weights[l]
    layers[l].bias = layers[l].bias - lr * nabla_biases[l]

Epoch progress: 100%|██████████| 60000/60000 [01:20<00:00, 747.56it/s]


Run the following code to make a forward pass over the test to see how it performed.

In [ ]:
# create a forward function for evaluation, without backprop
def forward(layers, x):
  """
  Forward propagation.
  """
  activation = x
  for layer in layers[1:]: #since layers[0] does not exist
    activation = sigmoid(layer.weight @ activation +layer.bias)
  return activation

# Evaluate
test_results = []
for (x,y) in val_loader:
  x = x.numpy().T
  y = y.numpy().T
  test_results.append((np.argmax(forward(layers, x)), y))

accuracy = sum(int(x == y) for (x, y) in test_results)/len(test_results)*100
print(accuracy)

8.91


<ipython-input-32-032826d20837>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  accuracy = sum(int(x == y) for (x, y) in test_results)/len(test_results)*100


And we can visualize some of our predictions with the code below

In [ ]:
# and show an example
x, y = next(iter(val_loader))
x = x.numpy().T
y = y.numpy().T
plt.title(f'Prediction is {np.argmax(forward(layers, x))}')
plt.imshow(x.reshape(28,28), cmap='gray');